In [8]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup
import time
import json 

import extracao

In [3]:
df = pd.read_excel("../Base 5.1.xlsx", sheet_name="Bloqueado_com_Resumo") 
df

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo
0,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2024,Relatório de pesquisa,Pesquisa,"coelho, daphne, teodorovicz, thomaz, fritscher...",Bloqueado,text/html; charset=utf-8,True
1,Sérgio Firpo,Estudos Raciais,https://repositorio.insper.edu.br/entities/pub...,2017,Capítulo de Livro,Livros,Sérgio Pinheiro Firpo et al.,Bloqueado,text/html; charset=utf-8,True
2,Sérgio Lazzarini,Metricis,https://repositorio.insper.edu.br/entities/pub...,2018,Capítulo de Livro,Livros,"marco antonio cesar bonomo, brito, ricardo d.,...",Bloqueado,text/html; charset=utf-8,True
3,Isabela Brandão Furtado,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Capítulo de Livro,Livros,Isabela Brandão Furtado; Carolina Pedrosa de Melo,Bloqueado,text/html; charset=utf-8,True


In [ ]:
def inicializa_driver():
    # A classe Service é usada para iniciar uma instância do Chrome Webdriver
    service = Service()

    # webdriver.ChromeOptions é usada para definir a preferência para o browser do chrome
    options = webdriver.ChromeOptions()

    # Inicia-se a instância do Chrome Webdriver com as definidas "options" e "service"
    return webdriver.Chrome(service=service, options=options)


driver = inicializa_driver()
lista = []

for url in df["Trabalho"]:
    driver.get(url)
    time.sleep(1.0)
    html = driver.page_source

    soup = BeautifulSoup(html, "html.parser")

    # Remover elementos que poluem o texto (navegação, footer, scripts etc.)
    for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "svg"]):
        tag.decompose()

    text = " ".join(soup.get_text(separator=" ").split())
    data = extracao.extracao_langchain(text)
    data_dict = data.model_dump()
    lista.append(json.dumps(data_dict, ensure_ascii=False, indent=2, default=extracao.enum_encoder))
    

In [ ]:
dados = [json.loads(s) for s in lista]


with open("resultados.json", "w", encoding="utf-8") as f:
    json.dump(dados, f, ensure_ascii=False, indent=2)